<a href="https://colab.research.google.com/github/hubertwel/paragraph-similarity/blob/main/paragraph-similarity/paragraph_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# UNDER CONSTRUCTION
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.corpora import Dictionary
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags, strip_multiple_whitespaces
from gensim.parsing.preprocessing import remove_stopwords, strip_short
from gensim.parsing.preprocessing import strip_non_alphanum, split_alphanum
from gensim.parsing.preprocessing import strip_punctuation
from gensim.utils import simple_preprocess
from gensim.utils import RULE_KEEP, RULE_DISCARD, RULE_DEFAULT
from gensim.test.utils import get_tmpfile
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize
import numpy as np
import os
import smart_open
import csv
import collections
import random
import re
import optuna

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        csv_reader = csv.DictReader(f, quoting=csv.QUOTE_ALL)
        CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_multiple_whitespaces]
        for i, row in enumerate(csv_reader):
          line = row['x']
          line = remove_urls(line)
          line_list = preprocess_string(line, CUSTOM_FILTERS)
          line = " ".join(line_list)
          tokens = simple_preprocess(line)
          if tokens_only:
            yield tokens
          else:
            # For training data, add tags
            yield TaggedDocument(tokens, [i])

def remove_urls(text):
  text = re.sub(r'https?:\/\/.*[\r\n]*', ' ', text, flags=re.MULTILINE)
  return text

def trim_rule(word, count, min_count):
    stop_words = set(stopwords.words('english')) 
    # This rule is only used to prune vocabulary during the current method call,
    # so that documents can be printed with stopwords and with words of any length
    if ((word in stop_words) or (len(word) < 3)):
        return utils.RULE_DISCARD  # throw out
    else:
        return utils.RULE_DEFAULT  # apply default rule, i.e. min_count

# Define an objective function to be maximized
def objective(trial):
  # Optimize hyperparameters:
  classifier_name = trial.suggest_categorical("classifier", ["LogReg"])
  if classifier_name == 'LogReg':
    logreg_c = trial.suggest_float("logreg_c", 1e-10, 1e10, log=True)
    clf = linear_model.LogisticRegression(C=logreg_c)
  # Scoring method:
    k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
    score = cross_val_score(clf, X_train, y_train, cv=k_fold, n_jobs=-1, scoring='accuracy')
    accuracy = score.mean()
    return accuracy

def main():    
  global X_train, y_train

  # Set file names for train and test data
  test_data_dir = os.path.join('/content/drive/MyDrive/Colab Notebooks/data/', 'gouvfr', 'CorpusRandomTwitter')
  print("test_data_dir: %s" % test_data_dir)
  train_file = os.path.join(test_data_dir, 'randomtweets3.txt')
  test_file = os.path.join(test_data_dir, 'randomtweets4.txt')

  train_corpus = list(read_corpus(train_file, tokens_only=True))
  train_corpus_tagged = list(read_corpus(train_file))
  test_corpus = list(read_corpus(test_file, tokens_only=True))

  print('train_corpus (the beginning): ', train_corpus[:2])
  print('train_corpus length %d' % len(train_corpus))
  print('train_corpus_tagged (the beginning): ', train_corpus_tagged[:2])
  print('test_corpus (the beginning): ', test_corpus[:2])
  print('test_corpus length %d' %len(test_corpus))
  print()

  # Build the model and its vocabulary
  model = Doc2Vec(dm=0, vector_size=80, min_count=3, epochs=50, hs=1, dbow_words=1, trim_rule=trim_rule)
  model.build_vocab(train_corpus_tagged)

  # Train the model on the train corpus
  model.train(train_corpus_tagged, total_examples=model.corpus_count, epochs=model.epochs)
  print()

  # Assess the model on the train corpus
  ranks = []
  first_ranks = []
  second_ranks = []
  inferred_vectors = []
  for doc_id in range(len(train_corpus_tagged)):
    inferred_vector = model.infer_vector(train_corpus_tagged[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    # sanity check (self-similarity)
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    first_ranks.append(sims[0][0])
    inferred_vectors.append(inferred_vector)
        
  print('sims: ', sims[:3])
  print('ranks: ', ranks)
  counter = collections.Counter(ranks)
  print(counter)
  print()

  # Test the model with one random document
  # Pick a random document from the test corpus and infer a vector from the model
  doc_id = random.randint(0, len(test_corpus) - 1)
  inferred_vector = model.infer_vector(test_corpus[doc_id])
  sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
  print('RANDOM TEST DOCUMENT ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))

  # Compare and print the most/second-most/third-most/median/least similar documents from the train corpus
  print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
  for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('THIRD-MOST', 2), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus_tagged[sims[index][0]].words)))

  # Save the model
  filename = get_tmpfile("parsim_doc2vec_model")
  model.save(filename)

  # Assess the model on the independent data set (test corpus)
  ranks_test = []
  first_ranks_test = []
  inferred_vectors_test = []
  for doc_id in range(len(test_corpus)):
    inferred_vector_test = model.infer_vector(test_corpus[doc_id])
    sims_test = model.dv.most_similar([inferred_vector_test], topn=len(model.dv))
    first_ranks_test.append(sims_test[0][0])
    inferred_vectors_test.append(inferred_vector_test)

  print('first_ranks_test: ', first_ranks_test)

  # Prepare vectors for cross validatiom
  tags_array_train = np.array(first_ranks)
  vectors_2Darray_train = np.array(inferred_vectors)
  tags_array_test = np.array(first_ranks_test)
  vectors_2Darray_test = np.array(inferred_vectors_test)
  y_train, X_train = tags_array_train, vectors_2Darray_train
  y_test, X_test = tags_array_test, vectors_2Darray_test

  # Create Optuna study
  study = optuna.create_study(direction="maximize")
  study.optimize(objective, n_trials=20)
  
  # Cross validation
  clf = LogisticRegression(solver='liblinear', max_iter=300, class_weight='balanced', multi_class='auto')
  k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
  score = cross_val_score(clf, X_train, y_train, cv=k_fold, n_jobs=-1, scoring='accuracy')
  print('score: ', score)
  print('Validation accuracy: {}'.format(round(np.mean(score)*100, 3)))
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print("Test accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))

main()

test_data_dir: /content/drive/MyDrive/Colab Notebooks/data/gouvfr/CorpusRandomTwitter
train_corpus (the beginning):  [['rt', 'americanidol', 'watch', 'as', 'ddlovato', 'gives', 'an', 'amazing', 'performance', 'of', 'her', 'single', 'stonecold', 'demionidol'], ['rt', 'dipti_varun', 'varun', 'dhawan', 'junaid', 'is', 'rookie', 'cop', 'and', 'heâ', 'kind', 'of', 'discovering', 'himself', 'as', 'the', 'movie', 'goes', 'on', 'varun_dvn', 'dishoom']]
train_corpus length 1000
train_corpus_tagged (the beginning):  [TaggedDocument(words=['rt', 'americanidol', 'watch', 'as', 'ddlovato', 'gives', 'an', 'amazing', 'performance', 'of', 'her', 'single', 'stonecold', 'demionidol'], tags=[0]), TaggedDocument(words=['rt', 'dipti_varun', 'varun', 'dhawan', 'junaid', 'is', 'rookie', 'cop', 'and', 'heâ', 'kind', 'of', 'discovering', 'himself', 'as', 'the', 'movie', 'goes', 'on', 'varun_dvn', 'dishoom'], tags=[1])]
test_corpus (the beginning):  [['live', 'morning', 'news', 'weather', 'and', 'traffic', 'upd

[I 2021-02-16 00:10:55,575] A new study created in memory with name: no-name-5aafc2d2-b60d-45c9-8551-8e6a5020e65f


first_ranks_test:  [128, 234, 542, 492, 994, 288, 349, 529, 238, 131, 168, 37, 814, 656, 662, 46, 582, 325, 87, 77, 142, 893, 208, 412, 83, 82, 312, 554, 507, 235, 108, 36, 647, 42, 494, 7, 145, 496, 262, 177, 375, 893, 686, 914, 485, 355, 949, 367, 76, 82, 106, 250, 535, 665, 347, 68, 954, 298, 389, 789, 621, 5, 82, 939, 64, 545, 738, 67, 205, 251, 781, 764, 96, 611, 516, 345, 285, 76, 131, 546, 994, 111, 131, 854, 503, 82, 186, 844, 642, 897, 87, 733, 221, 796, 768, 924, 131, 481, 147, 148, 634, 500, 131, 17, 391, 103, 141, 906, 657, 514, 791, 106, 95, 163, 46, 921, 117, 381, 956, 769, 647, 886, 859, 197, 314, 303, 131, 131, 608, 764, 974, 788, 165, 9, 576, 352, 557, 683, 438, 660, 457, 264, 226, 722, 715, 633, 11, 334, 201, 859, 607, 131, 656, 14, 659, 776, 682, 79, 87, 287, 259, 682, 399, 895, 352, 698, 259, 989, 383, 397, 621, 261, 964, 640, 720, 278, 992, 360, 775, 967, 628, 42, 82, 169, 199, 854, 909, 454, 426, 571, 474, 888, 633, 127, 318, 170, 234, 604, 430, 369, 795, 133, 635

[I 2021-02-16 00:11:37,170] Trial 0 finished with value: 0.11600000000000002 and parameters: {'classifier': 'LogReg', 'logreg_c': 55.084002559521835}. Best is trial 0 with value: 0.11600000000000002.
[I 2021-02-16 00:12:17,520] Trial 1 finished with value: 0.11800000000000002 and parameters: {'classifier': 'LogReg', 'logreg_c': 168.64771504476758}. Best is trial 1 with value: 0.11800000000000002.
[I 2021-02-16 00:12:49,186] Trial 2 finished with value: 0.12000000000000002 and parameters: {'classifier': 'LogReg', 'logreg_c': 180188539.01945555}. Best is trial 2 with value: 0.12000000000000002.
[I 2021-02-16 00:13:22,065] Trial 3 finished with value: 0.03400000000000001 and parameters: {'classifier': 'LogReg', 'logreg_c': 3.2018256198447797e-06}. Best is trial 2 with value: 0.12000000000000002.
[I 2021-02-16 00:13:52,863] Trial 4 finished with value: 0.03400000000000001 and parameters: {'classifier': 'LogReg', 'logreg_c': 1.2077337831279332e-06}. Best is trial 2 with value: 0.12000000000

score:  [0.14 0.13 0.13 0.14 0.12 0.06 0.09 0.15 0.09 0.11]
Validation accuracy: 11.6
Test accuracy: 0.527


In [ ]:
pip install -U scikit-learn

     |████████████████████████████████| 22.2MB 1.4MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
pip install optuna

     |████████████████████████████████| 296kB 9.0MB/s 
     |████████████████████████████████| 1.1MB 13.2MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 481kB 27.4MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 112kB 44.2MB/s 
     |████████████████████████████████| 143kB 34.9MB/s 
  Created wheel for alembic: filename=alembic-1.5.4-py2.py3-none-any.whl size=156314 sha256=8e4fe235d15e18da52fa546469aea999107635acb9f330e1ced84f013cf32ad9
  Stored in directory: /root/.cache/pip/wheels/fd/2d/ec/5a1b1e2363ed68392d292d215facf588d5448198edd8078bc1
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sha256=adde8992a87ec5a3a742b8dc7abad4ad5b18bcd35f53be76a3421346dc3e2e77
  Stored in directory: /root/.cache/pip/wheels/ad/10/d3/aeb26e20d19045e2a68e5d3cbb57432e11b5d9c92c99f98d47
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13702 sha

In [ ]:
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score
diabetes = datasets.load_diabetes()
X = diabetes.data[:150]
y = diabetes.target[:150]
print('X: ', X)
print('y: ', y)
y = diabetes.target[:150]
lasso = linear_model.Lasso()
print(cross_val_score(lasso, X, y, cv=3))


X:  [[ 0.03807591  0.05068012  0.06169621 ... -0.00259226  0.01990842
  -0.01764613]
 [-0.00188202 -0.04464164 -0.05147406 ... -0.03949338 -0.06832974
  -0.09220405]
 [ 0.08529891  0.05068012  0.04445121 ... -0.00259226  0.00286377
  -0.02593034]
 ...
 [-0.05637009 -0.04464164  0.09295276 ...  0.02545259  0.02605609
   0.04034337]
 [-0.06000263  0.05068012  0.01535029 ... -0.00259226 -0.03075121
  -0.0010777 ]
 [-0.04910502  0.05068012 -0.00512814 ...  0.07120998  0.06123791
  -0.03835666]]
y:  [151.  75. 141. 206. 135.  97. 138.  63. 110. 310. 101.  69. 179. 185.
 118. 171. 166. 144.  97. 168.  68.  49.  68. 245. 184. 202. 137.  85.
 131. 283. 129.  59. 341.  87.  65. 102. 265. 276. 252.  90. 100.  55.
  61.  92. 259.  53. 190. 142.  75. 142. 155. 225.  59. 104. 182. 128.
  52.  37. 170. 170.  61. 144.  52. 128.  71. 163. 150.  97. 160. 178.
  48. 270. 202. 111.  85.  42. 170. 200. 252. 113. 143.  51.  52. 210.
  65. 141.  55. 134.  42. 111.  98. 164.  48.  96.  90. 162. 150. 279.
  9

In [3]:
pip install --pre --upgrade gensim

     |████████████████████████████████| 24.0MB 72.7MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
